In [1]:
from song_data import SongData 

In [2]:
data = SongData('../data/chords_clean.csv')

In [3]:
df = data.df

In [4]:
df['chords'] = df['chords'].apply(eval) 

In [5]:
all_chords = []

for chord_list in df['chords']:
    all_chords.extend(chord_list)
    
len(all_chords)

349427

In [6]:
unique_chords = set([])

for chord_list in df['chords']:
    unique_chords = unique_chords.union(set(chord_list))
    
unique_chords = list(unique_chords)  
len(unique_chords)

1154

In [7]:
from pychord_jl import Chord

In [8]:
def replace_german_system(input):
    if input[0] == 'H':
        return 'Bb'+input[1:]
    
    return input

def replace_6_9(input):
    return (input
            .replace('6/9', '6_9')
            .replace('7/9', '7_9')
            .replace('7/5', '7_5')
           )

def replace_enharmonic(input):
    return (input
            .replace('E#', 'F')
            .replace('B#', 'C')
            .replace('Fb', 'E')
            .replace('Cb', 'B')
           )

def replace_errors(input):
    return (input
            .replace('G(2)', 'G')
            .replace('A*', 'A')
            .replace('(III)', '')
           )

def convert(input):
    input = replace_german_system(input)
    input = replace_6_9(input)
    input = replace_enharmonic(input)
    input = replace_errors(input)
    
    try:
        return Chord(input)
    except:
        return None

In [9]:
convert('D6/9')

<Chord: D6_9>

In [10]:
black_list = ['Go', 'AM']

In [11]:
parsed_chords = [ (convert(chord_str),chord_str) for chord_str in unique_chords if chord_str not in black_list]

In [12]:
def count_appearances(chord_str):
    return len([chord for chord in all_chords if chord == chord_str])

In [13]:
not_found = [ (count_appearances(x[1]),x[1]) for x in parsed_chords if x[0] is None]
len(not_found)

88

In [14]:
import functools 
total_not_found = functools.reduce(lambda x,y:x+y, [ count_appearances(x[1]) for x in parsed_chords if x[0] is None])

In [15]:
total_not_found

213

In [16]:
not_found.sort(key = lambda x: x[0], reverse = True) 

In [17]:
Chord('C#/F')

<Chord: C#/F>

In [18]:
not_found

[(8, 'C#b7b9'),
 (7, 'E7/9b'),
 (7, 'Fmaj7#11'),
 (6, 'B7/13b'),
 (6, 'B7/13'),
 (5, 'C#9(#5)'),
 (5, 'D#(b5)'),
 (4, 'F(5)'),
 (4, 'G7s4'),
 (4, 'F9/11'),
 (4, 'D7m'),
 (4, 'F#dim(F)'),
 (4, 'Gm7b5b9'),
 (4, 'Eaddb13'),
 (4, 'Em7add11'),
 (4, 'FmMaj7'),
 (4, 'Cm#'),
 (4, 'Dm7add11'),
 (4, 'B-'),
 (4, 'D(b5)'),
 (3, 'G6add11/B'),
 (3, 'Bmadd4'),
 (3, 'C7#5#9'),
 (3, 'G6add11'),
 (3, 'C3'),
 (3, 'DMin'),
 (3, 'BB'),
 (3, 'Emaj7b13/Ab'),
 (3, 'A7/6'),
 (3, 'Gm#'),
 (3, 'F#add6'),
 (2, 'E7(#5)'),
 (2, 'F#m7#5'),
 (2, 'D-'),
 (2, 'Em7/9'),
 (2, 'AmMaj7'),
 (2, 'RE'),
 (2, 'G#addb13'),
 (2, 'C(9)'),
 (2, 'C(maj7)'),
 (2, 'G-'),
 (2, 'A(major)'),
 (2, 'Bm7/9'),
 (2, 'Am7add11'),
 (2, 'Eb(b5)'),
 (2, 'F#m7(b5)'),
 (2, 'B6add11'),
 (2, 'G7(#5)'),
 (2, 'Eb7(b5)'),
 (2, 'Am7+'),
 (2, 'A3'),
 (2, 'B7#5#9'),
 (2, 'C#m7(b5)'),
 (2, 'C#mMaj7'),
 (2, 'E7(b13)'),
 (2, 'Fm#'),
 (2, 'Bb-'),
 (2, 'Db(b5)'),
 (1, 'B5b'),
 (1, 'Amb'),
 (1, 'Ebmaj7(#5)'),
 (1, 'A(maj)'),
 (1, 'Bb7(b13)'),
 (1, 'Bb7m'),
 (1,

In [19]:
def any_not_found(chords):
    return len([chord for chord in not_found if chord[1] in chords]) > 0

In [20]:
any_not_found(['A'])

False

In [21]:
any_not_found(['Go'])

False

In [22]:
len(df[df['chords'].apply(lambda x: any_not_found(x))]['url'])

73

In [23]:
(df[df['chords'].apply(lambda x: 'Cm(III)' in x)]['url']).iloc[0]

'https://tabs.ultimate-guitar.com/tab/eva-cassidy/over-the-rainbow-chords-64503'

In [24]:
Chord('A7/G').components()

['G', 'A', 'C#', 'E', 'G']